In [1]:
# declarative base

In [2]:
from sqlalchemy import create_engine
engine=create_engine("sqlite:///ex2.db", echo=True)

In [3]:
from sqlalchemy.ext.declarative import declarative_base # Meta Data를 대체
base=declarative_base()

In [4]:
# Metadata Class에 등록된(생성한) table 객체 초기화할 때 (혹시 잘못 typing했을 때)
# base.metadata.clear()

In [5]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
class User(base): # base 상속 받기
    __tablename__="User"
    
    uid=Column('uid', Integer, primary_key=True)
    name=Column('name',String, nullable=False)
    fullname=Column(String)
    password=Column("pass",String, nullable=False)
    
    def __repr__(self): # class print 했을 때 출력되는 애들
        return 'UID={0}, NAME={1}, PASS={2}'.format(
            self.uid, self.name, self.password)

In [6]:
# Create Table # in memory Table 객체 -> 실제 database Table로 생성
base.metadata.create_all(engine) # local에 file 생김 

2020-07-15 12:00:34,765 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 12:00:34,769 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:00:34,772 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 12:00:34,773 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:00:34,776 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("User")
2020-07-15 12:00:34,778 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:00:34,780 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "User" (
	uid INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	fullname VARCHAR, 
	pass VARCHAR NOT NULL, 
	PRIMARY KEY (uid)
)


2020-07-15 12:00:34,782 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:00:34,787 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
user1=User(name='test',password='testtest')

In [8]:
user1 # table instance created, 아직 primary key UID=0

UID=None, NAME=test, PASS=testtest

In [9]:
from sqlalchemy.orm import sessionmaker

In [10]:
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
session.add(user1) # add to Session, persisted to the database when 'commit'

In [12]:
session.commit() # added to DB

2020-07-15 12:00:41,006 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:00:41,009 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (name, fullname, pass) VALUES (?, ?, ?)
2020-07-15 12:00:41,011 INFO sqlalchemy.engine.base.Engine ('test', None, 'testtest')
2020-07-15 12:00:41,014 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
user1.uid # uid 값이 할당 됨

2020-07-15 12:00:42,478 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:00:42,480 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name", "User".fullname AS "User_fullname" 
FROM "User" 
WHERE "User".uid = ?
2020-07-15 12:00:42,483 INFO sqlalchemy.engine.base.Engine (1,)


1

In [14]:
user1.name = "개똥이" # user1 name 설정

In [15]:
# Update
session.dirty, session.is_modified # if modified, returns True.

(IdentitySet([UID=1, NAME=개똥이, PASS=testtest]),
 <bound method Session.is_modified of <sqlalchemy.orm.session.Session object at 0x000001651F0872B0>>)

In [16]:
session.commit() # log 찍어보면 UPDATE "User" SET name

2020-07-15 12:00:59,122 INFO sqlalchemy.engine.base.Engine UPDATE "User" SET name=? WHERE "User".uid = ?
2020-07-15 12:00:59,124 INFO sqlalchemy.engine.base.Engine ('개똥이', 1)
2020-07-15 12:00:59,127 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
user1 # NAME=개똥이 변경 반영됨

2020-07-15 12:01:00,015 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:01:00,017 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name", "User".fullname AS "User_fullname" 
FROM "User" 
WHERE "User".uid = ?
2020-07-15 12:01:00,018 INFO sqlalchemy.engine.base.Engine (1,)


UID=1, NAME=개똥이, PASS=testtest

In [18]:
# 한꺼번에 많이 add
session.add_all([User(name='test2',password='asdf'),
                User(name='test3',password='12345')])

In [19]:
cur=session.query(User)
for row in cur:
    print(row)
# [_ for _ in cur] 해도 같은 결과

2020-07-15 12:01:03,680 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (name, fullname, pass) VALUES (?, ?, ?)
2020-07-15 12:01:03,681 INFO sqlalchemy.engine.base.Engine ('test2', None, 'asdf')
2020-07-15 12:01:03,685 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (name, fullname, pass) VALUES (?, ?, ?)
2020-07-15 12:01:03,687 INFO sqlalchemy.engine.base.Engine ('test3', None, '12345')
2020-07-15 12:01:03,689 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name", "User".fullname AS "User_fullname" 
FROM "User"
2020-07-15 12:01:03,691 INFO sqlalchemy.engine.base.Engine ()
UID=1, NAME=개똥이, PASS=testtest
UID=2, NAME=test2, PASS=asdf
UID=3, NAME=test3, PASS=12345


In [20]:
[_ for _ in cur.filter(User.name=='test2')]

2020-07-15 12:01:05,883 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name", "User".fullname AS "User_fullname" 
FROM "User" 
WHERE "User".name = ?
2020-07-15 12:01:05,884 INFO sqlalchemy.engine.base.Engine ('test2',)


[UID=2, NAME=test2, PASS=asdf]

In [21]:
[_ for _ in cur.filter(User.name.like('test%'))]

2020-07-15 12:01:10,125 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name", "User".fullname AS "User_fullname" 
FROM "User" 
WHERE "User".name LIKE ?
2020-07-15 12:01:10,127 INFO sqlalchemy.engine.base.Engine ('test%',)


[UID=2, NAME=test2, PASS=asdf, UID=3, NAME=test3, PASS=12345]

In [22]:
base.metadata.clear()

In [23]:
class Artist(base):
    __tablename__='ARTIST'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Genre(base):
    __tablename__='GENRE'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Album(base):
    __tablename__='ALBUM'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('ARTIST.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK:{2}'.format(self.pk, self.name, self.fk)
    
class Track(base):
    __tablename__='Track'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk1 = Column('FK1', Integer, ForeignKey('ALBUM.ID'))
    fk2 = Column('FK2', Integer, ForeignKey('GENRE.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK1:{2}, FK2:{3}'.format(self.pk, self.name, self.fk1, self.fk2)

In [24]:
session.commit()

2020-07-15 12:01:17,894 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
base.metadata.create_all(engine)

2020-07-15 12:01:18,526 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ARTIST")
2020-07-15 12:01:18,529 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:01:18,531 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("GENRE")
2020-07-15 12:01:18,533 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:01:18,534 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ALBUM")
2020-07-15 12:01:18,535 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:01:18,536 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2020-07-15 12:01:18,537 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:01:18,539 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "ARTIST" (
	"ID" INTEGER NOT NULL, 
	"NAME" VARCHAR, 
	PRIMARY KEY ("ID")
)


2020-07-15 12:01:18,540 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:01:18,544 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-15 12:01:18,546 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "GENRE" (
	"ID" INTEGER NOT NULL, 
	"NAME" VARCHAR, 
	P

In [26]:
# Insert
artist1=Artist(name="Led Zepplin")
artist2=Artist(name="AC/DC")

session.add_all([artist1, artist2])
session.commit()

album=[Album(name="IV", fk=artist1.pk),
      Album(name="Who Made Who", fk=artist2.pk)]

session.add_all(album)
session.commit()

session.add_all([Genre(name="Rock"), Genre(name="Metal")])
session.commit()

2020-07-15 12:01:24,045 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:01:24,047 INFO sqlalchemy.engine.base.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2020-07-15 12:01:24,048 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2020-07-15 12:01:24,052 INFO sqlalchemy.engine.base.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2020-07-15 12:01:24,053 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2020-07-15 12:01:24,055 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-15 12:01:24,067 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:01:24,069 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 12:01:24,070 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 12:01:24,072 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 12:01:24,074 INFO sqlal

In [27]:
album1=session.query(Album).filter(Album.fk==artist1.pk).one()
album2=session.query(Album).filter(Album.fk==artist2.pk).one()

genre1=session.query(Genre).filter(Genre.name=="Rock").filter(Genre.pk==1).one()
genre2=session.query(Genre).filter(Genre.name=="Metal").filter(Genre.pk==2).one()

2020-07-15 12:01:28,078 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:01:28,079 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 12:01:28,080 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 12:01:28,083 INFO sqlalchemy.engine.base.Engine SELECT "ALBUM"."ID" AS "ALBUM_ID", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."FK" = ?
2020-07-15 12:01:28,084 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 12:01:28,087 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 12:01:28,088 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 12:01:28,090 INFO sqlalchemy.engine.base.Engine SELECT "ALBUM"."ID" AS "ALBUM_ID", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."FK" = ?
2020-07-15 1

In [28]:
session.add_all([
    Track(name='track1', fk1=album1.pk, fk2=genre1.pk),
    Track(name='track2', fk1=album2.pk, fk2=genre2.pk),
    Track(name='track3', fk1=album1.pk, fk2=genre1.pk),
    Track(name='track4', fk1=album2.pk, fk2=genre2.pk)
])
session.commit()

2020-07-15 12:01:30,980 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 12:01:30,982 INFO sqlalchemy.engine.base.Engine ('track1', 1, 1)
2020-07-15 12:01:30,986 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 12:01:30,988 INFO sqlalchemy.engine.base.Engine ('track2', 2, 2)
2020-07-15 12:01:30,990 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 12:01:30,992 INFO sqlalchemy.engine.base.Engine ('track3', 1, 1)
2020-07-15 12:01:30,993 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 12:01:30,994 INFO sqlalchemy.engine.base.Engine ('track4', 2, 2)
2020-07-15 12:01:30,996 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
# Join
result = session.query(Track.name, Album.name, Genre.name, Artist.name) \
            .select_from(Track) \
            .join(Album)\
            .join(Genre)\
            .join(Artist).all()

for row in result:
    print(row)

2020-07-15 12:01:33,183 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:01:33,185 INFO sqlalchemy.engine.base.Engine SELECT "Track"."NAME" AS "Track_NAME", "ALBUM"."NAME" AS "ALBUM_NAME", "GENRE"."NAME" AS "GENRE_NAME", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "Track" JOIN "ALBUM" ON "ALBUM"."ID" = "Track"."FK1" JOIN "GENRE" ON "GENRE"."ID" = "Track"."FK2" JOIN "ARTIST" ON "ARTIST"."ID" = "ALBUM"."FK"
2020-07-15 12:01:33,186 INFO sqlalchemy.engine.base.Engine ()
('track1', 'IV', 'Rock', 'Led Zepplin')
('track2', 'Who Made Who', 'Metal', 'AC/DC')
('track3', 'IV', 'Rock', 'Led Zepplin')
('track4', 'Who Made Who', 'Metal', 'AC/DC')


In [30]:
songList = [dict(Track=row[0], Album=row[1], Genre=row[2], Artist=row[3]) for row in result]
songList

[{'Track': 'track1', 'Album': 'IV', 'Genre': 'Rock', 'Artist': 'Led Zepplin'},
 {'Track': 'track2',
  'Album': 'Who Made Who',
  'Genre': 'Metal',
  'Artist': 'AC/DC'},
 {'Track': 'track3', 'Album': 'IV', 'Genre': 'Rock', 'Artist': 'Led Zepplin'},
 {'Track': 'track4',
  'Album': 'Who Made Who',
  'Genre': 'Metal',
  'Artist': 'AC/DC'}]

In [94]:
# Relationship 
# pk-fk 연동인 두 class에게 서로 1:1, 1:N등으로 참조, 역참조할 수 있도록 해줌
# relationship btw two mapped classes
# table 다시 등록해야함

In [39]:
base.metadata.clear()

In [40]:
from sqlalchemy.orm import relationship

class Artist(base):
    __tablename__='Artist'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    
    albumlist = relationship("Artist", back_populates="artist")
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Genre(base):
    __tablename__='Genre'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    
    tracklist = relationship("Track", back_populates="album")
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Album(base):
    __tablename__='Album'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('ARTIST.ID'))
    
    artist = relationship("Artist", back_populates="albumlist", uselist=False)
    tracklist = relationship("Track", back_populates="album")
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK:{2}'.format(self.pk, self.name, self.fk)
    
class Track(base):
    __tablename__='Track'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk1 = Column('FK1', Integer, ForeignKey('ALBUM.ID'))
    fk2 = Column('FK2', Integer, ForeignKey('GENRE.ID'))
    
    album = relationship("Album", back_populates='tracklist', uselist=False)
    genre = relationship("Genre", back_populates='tracklist', uselist=False)
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK1:{2}, FK2:{3}'.format(self.pk, self.name, self.fk1, self.fk2)

In [41]:
session.commit()

In [42]:
track1 = Track(name='track1')
track2 = Track(name='track2')
track3 = Track(name='track3')
track4 = Track(name='track4')

track1.album= track2.album = Album(name="IV")
track3.album= track4.album = Album(name="Who Made Who")

track1.genre = track3.genre = Genre(name="Rock")
track2.genre = track4.genre = Genre(name="Metal")
                                    
track1.album.artist = track2.album.artist = Artist(name="Led Zepplin")
track3.album.artist = track4.album.artist = Artist(name="AC/DC")

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'mapped class Artist->ARTIST'. Original exception was: relationship 'albumlist' expects a class or a mapper argument (received: <class 'sqlalchemy.sql.schema.Table'>)